In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#Step1: Necessery imports
import pandas as pd
import numpy as np
import plotly as plt
import matplotlib as mpl
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from pandas import DataFrame


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Step2: Loading the data 
#Change the csv file for another brand
#Added Data types for memory optimization and faster loading
data = pd.read_csv('/kaggle/input/cat-vs-dog-popularity-in-us/catsvdogs.csv')
            
print(data.info()) 

# Pets Ownership -  Cats vs Dog Popularity in US

Population and ownership by household of dogs and cats broken down by state via American Veterinary Medical Association.

![](https://imagesvc.meredithcorp.io/v3/mm/gif?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F35%2F2016%2F11%2F04021528%2Fkittens-turn-table.gif)

![](https://imagesvc.meredithcorp.io/v3/mm/gif?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F35%2F2016%2F11%2F04021526%2Fpuppy-pile-gif.gif)


# Describing the data

In [ ]:
#Step3:Describing the data
data.describe()

# Percentage of households with pets

In [ ]:
fig = px.bar(data, y='Percentage of households with pets', x='Location')
fig.show();

In [ ]:
fig = px.pie(data, values='Percentage of households with pets', names='Location')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show();

# Percentage of Dog Owners 2021

In [ ]:
fig = px.bar(data, y='Percentage of Dog Owners', x='Location')
fig.show();

In [ ]:
fig = px.pie(data, values='Percentage of Dog Owners', names='Location')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show();

# Mean Number of Dogs per household


In [ ]:
fig = px.bar(data, y='Mean Number of Dogs per household', x='Location')
fig.show();

# Percentage of Cat Owners

In [ ]:
fig = px.bar(data, y='Percentage of Cat Owners', x='Location')
fig.show();

In [ ]:
fig = px.pie(data, values='Percentage of Cat Owners', names='Location')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show();

# Mean Number of Cats


In [ ]:
fig = px.bar(data, y='Mean Number of Cats', x='Location')
fig.show();

In [ ]:
fig = px.pie(data, values='Mean Number of Cats', names='Location')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show();

# Maps

In [ ]:
!pip install geopy==1.22.0
import math
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
from folium.plugins import MarkerCluster
from geopandas.tools import geocode
result = geocode("The Great Pyramid of Giza", provider="nominatim")
result

In [ ]:
point = result.geometry.iloc[0]
print("Latitude:", point.y)
print("Longitude:", point.x)

In [ ]:
def my_geocoder(row):
    try:
        point = geocode(row, provider='nominatim').geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        return None

data[['Latitude', 'Longitude', 'geometry']] = data.apply(lambda x: my_geocoder(x['Location']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(data["Latitude"])) / len(data)) * 100))

# Drop data that were not successfully geocoded
data = data.loc[~np.isnan(data["Latitude"])]
data = gpd.GeoDataFrame(data, geometry=data.geometry)
data.crs = {'init': 'epsg:4326'}
data.head()

In [ ]:
# Create the map
v_map = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2) 

# Add points to the map
mc = MarkerCluster()
for idx, row in data.iterrows(): 
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(Marker(location=[row['Latitude'], row['Longitude']],
                            tooltip=str(row['Percentage of Dog Owners'])+" Percentage of Dog Owners"))
v_map.add_child(mc)

#Style map
folium.TileLayer('stamentoner').add_to(v_map)

# Display the map
v_map

In [ ]:
# Create the map
v_map = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2) 

# Add points to the map
mc = MarkerCluster()
for idx, row in data.iterrows(): 
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(Marker(location=[row['Latitude'], row['Longitude']],
                            tooltip=str(row['Percentage of Cat Owners'])+" Percentage of Cat Owners"))
v_map.add_child(mc)

#Style map
folium.TileLayer('stamentoner').add_to(v_map)

# Display the map
v_map

# Finding Influences - Correlation Matrix (impact relationship with numbers)

In [ ]:
#Finding correlations

corr = data.corr()

mask = np.triu(np.ones_like(corr, dtype=np.bool))
corr = corr.mask(mask)
fig = ff.create_annotated_heatmap(
    z=corr.to_numpy().round(2),
    x=list(corr.index.values),
    y=list(corr.columns.values),       
    xgap=3, ygap=3,
    zmin=-1, zmax=1,
colorscale='icefire',
    colorbar_thickness=30,
    colorbar_ticklen=3,
)
fig.update_layout(title_text='Correlation Matrix (impact relationship with numbers)',
                  title_x=0.5,
                  titlefont={'size': 20},
                  width=800, height=800,
                  xaxis_showgrid=False,
                  xaxis={'side': 'bottom'},
                  yaxis_showgrid=False,
                  yaxis_autorange='reversed',                   
                  paper_bgcolor=None,
                  template="simple_white"
                  )
fig.show()

# Pairplots and Linear Predictions

In [ ]:
sns.pairplot(data,palette='bright')
%time

In [ ]:
sns.set(style="ticks", color_codes=True)
g = sns.pairplot(data, kind="reg", plot_kws={'line_kws':{'color':'red'}})
plt.show()
%time

# Pandas Profiling

In [ ]:
import pandas_profiling
report = pandas_profiling.ProfileReport(data)
from IPython.display import display
display(report)

# Quickda Analysis

In [ ]:
pip install quickda

In [ ]:
# Importing libraries
from quickda.explore_data import *
from quickda.clean_data import *
from quickda.explore_numeric import *
from quickda.explore_categoric import *
from quickda.explore_numeric_categoric import *
from quickda.explore_time_series import *

In [ ]:
eda_num(data)

In [ ]:
eda_num(data, method="correlation")